In [13]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
from seaborn import color_palette
import cv2

In [14]:
_BATCH_NORM_DECAY = 0.9
_BATCH_NORM_EPSILON = 1e-05
_LEAKY_RELU = 0.1

def batch_norm(input,format,learning):
    layers = tf.layers.batch_normalization(inputs = input,momentum = _BATCH_NORM_DECAY,epsilon =_BATCH_NORM_EPSILON ,training = learning, axis = 1 if format == 'channels_first' else 3)
    return layers

def padded_input(input,kernelsize,format):
    pad_s = (kernelsize - 1)//2
    pad_e = kernelsize - 1 - pad_s
    if format == 'channels_first':
        return tf.pad(input,[[0,0],[0,0],[pad_s,pad_e],[pad_s,pad_e]])
    else:
        return tf.pad(input,[[0,0],[pad_s,pad_e],[pad_s,pad_e],[0,0]])

def conv_layer_padding(input,stride,filter,kernel_size,format):
    if stride >1:
        input = padded_input(input,kernel_size,format)
        padding = 'VALID'
    else:
        padding = 'SAME'
    conv_output = tf.layers.conv2D(inputs = input,filters= filter,kernel_size = kernel_size,padding = padding,strides = stride,data_format = format)

def yolo_convolution_block(inputs, filters, training, data_format):
    inputs = conv_layer_padding(inputs, filters=filters, kernel_size=1,data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    inputs = conv_layer_padding(inputs, filters=2 * filters, kernel_size=3,data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    inputs = conv_layer_padding(inputs, filters=filters, kernel_size=1,data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    inputs = conv_layer_padding(inputs, filters=2 * filters, kernel_size=3,data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    inputs = conv_layer_padding(inputs, filters=filters, kernel_size=1,data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    route = inputs
    inputs = conv_layer_padding(inputs, filters=2 * filters, kernel_size=3,data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    return route, inputs